In [ ]:
import string

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup

# The Economist

In [ ]:
import requests
 
URL = "https://www.economist.com/economics-a-to-z/a"
r = requests.get(URL)

In [ ]:
soup = BeautifulSoup(r.content, 'html5lib') 

In [ ]:
li = soup.find_all('div', {"class": " ec-glossary-type node node-type-economics_term node-published node-teaser clearfix"})
dtt = [(i.find('h2').text, i.find('p').text) for i in li]

In [ ]:
pd.DataFrame(dtt, columns=['terms', 'links'])

,terms,links
0,Absolute advantage,This is the simplest yardstick of economic per...
1,Adaptive expectations,A theory of how people form their views about ...
2,Adverse selection,When you do business with people you would be ...
3,Advertising,"Many firms advertise their goods or services, ..."
4,Agency costs,These can arise when somebody (the principal) ...
5,Agricultural policy,Countries often provide support for their farm...
6,Agriculture,Farming around the world continues to become m...
7,Aid,See international aid.
8,Altruism,It is often alleged that altruism is inconsist...
9,Amortisation,The running down or payment of a loan by insta...


In [ ]:
all_econ = pd.DataFrame({})
base_url = "https://www.economist.com/economics-a-to-z/"
for ch in 'abcdefghijklmnopqrstuvwxyz':
  r = requests.get(base_url+ch)
  soup = BeautifulSoup(r.content, 'html5lib') 
  li = soup.find_all('div', {"class": " ec-glossary-type node node-type-economics_term node-published node-teaser clearfix"})
  dtt = [(i.find('h2').text, i.find('p').text) for i in li]
  all_econ = all_econ.append(pd.DataFrame(dtt, columns=['terms', 'definitions']))

In [ ]:
 all_econ['source'] = 'economist'

In [ ]:
!pip install textstat

     |████████████████████████████████| 101 kB 10.3 MB/s 
     |████████████████████████████████| 2.0 MB 23.6 MB/s 


In [ ]:
import textstat

In [ ]:
def extract_scores(test_data):
  try:
    scores = [textstat.flesch_reading_ease(test_data),textstat.flesch_kincaid_grade(test_data),textstat.smog_index(test_data),textstat.coleman_liau_index(test_data), textstat.automated_readability_index(test_data),textstat.dale_chall_readability_score(test_data), textstat.difficult_words(test_data), textstat.linsear_write_formula(test_data), textstat.gunning_fog(test_data), textstat.text_standard(test_data), textstat.fernandez_huerta(test_data), textstat.szigriszt_pazos(test_data), textstat.gutierrez_polini(test_data), textstat.crawford(test_data), textstat.gulpease_index(test_data), textstat.osman(test_data)]
  except:
    scores = [0.0]*9 + ['NOT USE'] + [0.0]*6
  return scores

In [ ]:
scores = all_econ['definitions'].apply(lambda x: extract_scores(str(x)))
scores_df = pd.DataFrame(list(scores))
scores_df.columns = ['flesch_reading_ease','flesch_kincaid_grade','smog_index','coleman_liau_index','automated_readability_index','dale_chall_readability_score','difficult_words','linsear_write_formula','gunning_fog','text_standard','fernandez_huerta','szigriszt_pazos','gutierrez_polini','crawford','gulpease_index','osman']
scores_df

,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard,fernandez_huerta,szigriszt_pazos,gutierrez_polini,crawford,gulpease_index,osman
0,53.61,10.2,13.3,12.93,12.8,8.28,19,12.000000,12.04,12th and 13th grade,92.9,88.71,38.00,3.3,52.181818,46.08
1,64.20,8.2,0.0,10.84,9.7,10.52,10,10.250000,12.65,10th and 11th grade,101.0,94.87,42.21,2.9,58.677419,59.47
2,54.15,12.0,14.4,10.80,14.3,7.94,40,13.200000,13.15,13th and 14th grade,90.9,86.02,39.87,3.3,51.834646,50.77
3,21.74,16.2,17.1,16.60,18.3,10.75,32,18.000000,16.68,16th and 17th grade,68.4,66.02,30.62,4.9,42.645833,25.55
4,46.44,15.0,14.6,11.67,18.8,9.42,18,20.500000,16.43,14th and 15th grade,83.2,81.33,36.23,3.2,47.787879,40.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,38.66,13.8,16.3,12.83,15.4,10.31,23,17.333333,16.27,15th and 16th grade,80.4,74.68,36.95,4.2,48.722222,41.42
652,63.70,8.4,13.0,7.54,7.5,8.05,8,11.000000,11.40,7th and 8th grade,100.5,94.79,47.57,2.9,63.375000,72.71
653,55.68,11.4,13.2,11.67,14.5,9.04,39,13.000000,13.12,12th and 13th grade,92.5,89.35,38.95,3.1,50.623037,49.58
654,59.33,10.0,12.7,11.26,12.4,8.79,35,11.000000,12.05,12th and 13th grade,96.1,93.22,40.33,2.9,53.426230,53.43


In [ ]:
all_econ_scores_df = pd.concat([all_econ.reset_index(),scores_df.reset_index()], axis = 1).drop('index', axis = 1)
all_econ_scores_df

,terms,definitions,source,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard,fernandez_huerta,szigriszt_pazos,gutierrez_polini,crawford,gulpease_index,osman
0,Absolute advantage,This is the simplest yardstick of economic per...,economist,53.61,10.2,13.3,12.93,12.8,8.28,19,12.000000,12.04,12th and 13th grade,92.9,88.71,38.00,3.3,52.181818,46.08
1,Adaptive expectations,A theory of how people form their views about ...,economist,64.20,8.2,0.0,10.84,9.7,10.52,10,10.250000,12.65,10th and 11th grade,101.0,94.87,42.21,2.9,58.677419,59.47
2,Adverse selection,When you do business with people you would be ...,economist,54.15,12.0,14.4,10.80,14.3,7.94,40,13.200000,13.15,13th and 14th grade,90.9,86.02,39.87,3.3,51.834646,50.77
3,Advertising,"Many firms advertise their goods or services, ...",economist,21.74,16.2,17.1,16.60,18.3,10.75,32,18.000000,16.68,16th and 17th grade,68.4,66.02,30.62,4.9,42.645833,25.55
4,Agency costs,These can arise when somebody (the principal) ...,economist,46.44,15.0,14.6,11.67,18.8,9.42,18,20.500000,16.43,14th and 15th grade,83.2,81.33,36.23,3.2,47.787879,40.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,X-efficiency,Producing OUTPUT at the minimum possible cost....,economist,38.66,13.8,16.3,12.83,15.4,10.31,23,17.333333,16.27,15th and 16th grade,80.4,74.68,36.95,4.2,48.722222,41.42
652,Yield,"The annual income from a SECURITY, expressed a...",economist,63.70,8.4,13.0,7.54,7.5,8.05,8,11.000000,11.40,7th and 8th grade,100.5,94.79,47.57,2.9,63.375000,72.71
653,Yield curve,Shorthand for comparisons of the INTEREST RATE...,economist,55.68,11.4,13.2,11.67,14.5,9.04,39,13.000000,13.12,12th and 13th grade,92.5,89.35,38.95,3.1,50.623037,49.58
654,Yield gap,A way of comparing the performance of BONDS an...,economist,59.33,10.0,12.7,11.26,12.4,8.79,35,11.000000,12.05,12th and 13th grade,96.1,93.22,40.33,2.9,53.426230,53.43


In [ ]:
all_econ_scores_df.to_csv('all_econ_scores_df.csv', index = False)

# Federal Reserve Bank St. Louis

In [ ]:
#https://www.stlouisfed.org/education/glossary#sr_education%20level=9--12
#https://www.stlouisfed.org/education/glossary#sr_education%20level=College
#https://www.stlouisfed.org/education/glossary#sr_education%20level=Non--educators
#https://www.stlouisfed.org/education/glossary#sr_education%20level=Pre--K--5
#https://www.stlouisfed.org/education/glossary#sr_education%20level=6--8

In [ ]:
def file_to_df(fl):
  lines = open(fl,'r').read().splitlines()
  terms, definitions = [],[]
  for i in range(len(lines)):
    if i%2==0:
      terms.append(lines[i])
    else:
      definitions.append(lines[i])
  df = pd.DataFrame({'terms':terms, 'definitions':definitions})
  df['source'] = fl.replace('.txt', '')

  scores = df['definitions'].apply(lambda x: extract_scores(str(x)))
  scores_df = pd.DataFrame(list(scores))
  scores_df.columns = ['flesch_reading_ease','flesch_kincaid_grade','smog_index','coleman_liau_index','automated_readability_index','dale_chall_readability_score','difficult_words','linsear_write_formula','gunning_fog','text_standard','fernandez_huerta','szigriszt_pazos','gutierrez_polini','crawford','gulpease_index','osman']

  df_scores_df = pd.concat([df.reset_index(),scores_df.reset_index()], axis = 1).drop('index', axis = 1)
  return df_scores_df

In [ ]:
six_eight = file_to_df('6_8_louis.txt')
nine_twelve = file_to_df('9_12_louis.txt')
college = file_to_df('college_louis.txt')
non_edu = file_to_df('non_educators_louis.txt')
pre_lou = file_to_df('pre_louis.txt')

In [ ]:
all_louis = six_eight.append(nine_twelve).append(college).append(non_edu).append(pre_lou)

In [ ]:
all_louis

,terms,definitions,source,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard,fernandez_huerta,szigriszt_pazos,gutierrez_polini,crawford,gulpease_index,osman
0,Absolute advantage,The ability to produce more of a good or servi...,6_8_louis,57.61,10.7,0.0,9.34,11.4,8.32,5,15.0,12.44,10th and 11th grade,94.4,88.55,42.97,3.2,56.272727,60.85
1,Adjustable-rate mortgage (ARM),A mortgage that permits the lender to periodic...,6_8_louis,41.70,12.7,0.0,10.33,11.7,12.20,8,13.5,14.11,12th and 13th grade,83.4,82.00,41.66,3.8,55.190476,57.42
2,Adjusted gross income,Gross income minus specific adjustments.,6_8_louis,15.64,12.3,0.0,18.88,15.0,16.52,3,3.5,18.00,12th and 13th grade,69.8,64.78,25.55,3.3,77.000000,21.61
3,Allocation,"Ways in which to distribute goods, services or...",6_8_louis,79.26,4.4,0.0,11.22,8.7,11.10,3,4.5,8.04,4th and 5th grade,113.7,107.85,41.39,1.4,67.888889,62.69
4,Alt-A (Alternative A) mortgage,A non-standard mortgage owed by a borrower cha...,6_8_louis,1.77,19.7,0.0,18.22,21.7,13.16,12,22.5,19.69,19th and 20th grade,53.3,50.62,26.83,6.0,37.148148,13.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Utility/Satisfaction,The total satisfaction received from consuming...,pre_louis,36.96,10.3,0.0,17.66,13.4,12.86,5,5.5,12.49,12th and 13th grade,83.7,80.16,30.62,3.6,57.888889,33.14
174,Wages (elementary),Income people receive for work that they do.,pre_louis,80.28,4.1,0.0,6.45,4.4,0.40,0,3.0,3.20,3rd and 4th grade,114.7,113.17,48.75,0.8,80.250000,80.83
175,Wants,Desires that can be satisfied by consuming goo...,pre_louis,61.33,7.2,0.0,11.40,8.5,10.45,4,6.0,12.00,8th and 9th grade,100.6,97.16,41.26,2.4,66.000000,60.06
176,Warranty,A guarantee to a consumer that promises a prod...,pre_louis,56.25,9.1,0.0,9.56,8.7,10.70,6,8.5,11.33,8th and 9th grade,95.6,92.16,44.04,3.1,61.000000,63.05


In [ ]:
all_louis.to_csv('all_louis_gradewise.csv', index = False)

# Palgrave

In [ ]:
!pip install PyPDF2

     |████████████████████████████████| 77 kB 5.6 MB/s 
ERROR: Operation cancelled by user


In [ ]:
import PyPDF2

ModuleNotFoundError: ignored

In [ ]:
def extract_text(pdf_name):
  content = ''

  # creating a pdf file object 
  pdfFileObj = open(pdf_name, 'rb')
  
  # creating a pdf reader object 
  pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
  for pg_no in range(pdfReader.getNumPages()):
    #print(pg_no)
    # creating a page object 
    pageObj = pdfReader.getPage(pg_no) 
    content = content + str(pageObj.extractText())
  pdfFileObj.close()
  return content

In [ ]:
palgrave = extract_text('The Palgrave Macmillan Dictionary of Finance, Investment and Banking by Erik Banks (auth.) (z-lib.org)-pages-35-587.pdf')

In [ ]:
palgrave

In [ ]:
# Palgrave pdf to html : using https://www.pdftohtml.net/ (not good) and https://pdf.online/convert-pdf-to-html (better)

In [ ]:
soup = BeautifulSoup(open("palgrave_better.html", "r"), 'html.parser')

In [ ]:
import re


In [ ]:
#soup.find_all('span', {"class": "ft2"})
soup.find_all('p', {"class": re.compile("\ft3$")})

In [ ]:
soup.find_all(re.compile("\ft3$"))

In [ ]:
soup.findAll(p=re.compile("\ft3$"))

In [ ]:
[i for i in soup.find_all('p', {'class' : re.compile('ft3$')})]

In [ ]:
li = soup.find_all('p', {'class' : re.compile('ft3$')})

In [ ]:
li[0].text

In [ ]:
li[0].text.split('</span>')

In [ ]:
term_defn = []
for cnt,i in enumerate(soup.find_all('p', {'class' : re.compile('ft3$')})):
  term = ' '.join([j.text.strip() for j in i.find_all('span', {'class':"ft2"})])
  all_text = i.text
  definition = all_text.replace(term, '').strip()
  if term.strip() != '':
    term_defn.append([term, definition])
  else:
    term_defn[-1][1] = term_defn[-1][1] + definition


In [ ]:
palgrave_df = pd.DataFrame(term_defn, columns = ['terms', 'definitions'])
palgrave_df['source'] = 'palgrave'

In [ ]:
palgrave_df.sample(60)

In [ ]:
scores = palgrave_df['definitions'].apply(lambda x: extract_scores(str(x)))
scores_df = pd.DataFrame(list(scores))
scores_df.columns = ['flesch_reading_ease','flesch_kincaid_grade','smog_index','coleman_liau_index','automated_readability_index','dale_chall_readability_score','difficult_words','linsear_write_formula','gunning_fog','text_standard','fernandez_huerta','szigriszt_pazos','gutierrez_polini','crawford','gulpease_index','osman']


In [ ]:
palgrave_scores_df = pd.concat([palgrave_df.reset_index(),scores_df.reset_index()], axis = 1).drop('index', axis = 1)
palgrave_scores_df

In [ ]:
palgrave_scores_df.to_csv('palgrave_scores_df.csv', index = False)

# Heather

In [ ]:
soup = BeautifulSoup(open("heather.html", "r"), 'html.parser')

In [ ]:
li = soup.find_all('p', {'class' : [re.compile('^p1 '), re.compile('^p0 ')]})

In [ ]:
'2'.isnumeric()

True

In [ ]:
#terms 2,10,11 18, 6, 13, 22, 20
terms = [i.text.strip() for i in soup.find_all('span', {'class':["ft2", "ft6","ft10","ft11", "ft13","ft18", "ft20", "ft22"]}) if i.text.strip()!='' and i.text.strip().replace('.','').replace('–','').replace('(','').replace(')','').isnumeric()==False]
terms_to_remove = ['noun ', ',', 'f.', 'name', 'A']
terms_use = [i for i in terms if i not in terms_to_remove]

In [ ]:
[i.text for i in soup.find_all('p')]

['A, AA, AAA noun letters indicating that a share o certain rating for reliability. The AAA rating (called t given by Standard & Poor’s or by Moody’s Investors Se very high level of reliability for a corporate or municipal',
 'ability to pay theory noun the theory of taxa taxation should be related to the taxpayer’s ability to higher incomes pay tax at a higher rate than those on low basis of progressive taxation. Many taxes, such as VAT, are not linked to the purchaser’s ability to pay and ar taxation.',
 'abscissa noun the horizontal value on a graph. Th y-value or ordinate. Also called x-value (NOTE: The pl',
 'abscissas.)',
 'absolute advantage, absolute cost advanta',
 'in which a country, or sometimes a person or company producing something than its competitors (i.e. its out higher). This gives an advantage to established firms low in comparison to new entrants.',
 'absolute value noun the size or value of a number The absolute value of –62.34 is 62.34.',
 'absorption costing, ab

In [ ]:
len(terms_use)

2366

In [ ]:
term_all, defn_all = [], []
n = 0
for  ind in range(len(terms_use)):
  term = terms_use[ind]
  for i in soup.find_all('p')[n:]:
    if term in i.text:
      term_all.append(term)
      defn_all.append(i.text.replace(term, '').replace('noun', '').replace('plural','').replace('adverb', '').replace('verb','').replace('abbreviation', '').strip())
      n = n + 1
      break



In [ ]:
len(defn_all)

2366

In [ ]:
heather_df = pd.DataFrame({'terms':term_all, 'definitions':defn_all})

In [ ]:
heather_df.sample(60)

,terms,definitions
11,monetary policy,"accommodating ,"
140,bear,banknote a promissory note issued by a bank e...
1955,share option,the right to buy or sell shares at time in the...
1365,nationalisation,", nationalization the tak industry by the state"
356,convexity,the convex shape of a curve. The the a set are...
1128,L/C,LBO leveraged buyout letter of credit
1100,kinked demand curve,a demand curve w believe that if they raise th...
218,capital equipment,equipment which a fac work or in production
521,payout ratio,"dividend cover, dividend profits which is pa..."
1084,issuing house,term is .)


In [ ]:
heather_df.to_csv('heather_df.csv', index = False)

# Heather : will do later

# Mankiw

In [ ]:
soup = BeautifulSoup(open("mankiw.html", "r"), 'html.parser')

In [ ]:
terms = [i.text for i in soup.find_all('span', {'class' : 'ft3'})]

In [ ]:
terms

In [ ]:
all_texts = [i.text for i in soup.find_all('p') if 'Cengage Learning' not in i.text and 'Editorial review' not in i.text]

In [ ]:
use_terms = []
use_definitions = []
n = 0
for term in terms:
  for txt in all_texts[n:]:
    if term in txt:
      use_terms.append(term)
      use_definitions.append(txt.split('\u2002')[1])
      n = n + 1

In [ ]:
mankiw = pd.DataFrame({'terms':use_terms, 'definitions':use_definitions})
mankiw['source'] = 'mankiw_copyright'
mankiw

,terms,definitions,source
0,absolute advantage,the ability to produce a good using fewer inp...,mankiw_copyright
1,accounting profit,total revenue minus total explicit cost,mankiw_copyright
2,adverse selection,the tendency for the mix of unobserved attrib...,mankiw_copyright
3,agent,a person who is performing an act for another...,mankiw_copyright
4,agent,"a person for whom another person, called the ...",mankiw_copyright
...,...,...,...
294,welfare,government programs that supplement the incom...,mankiw_copyright
295,welfare,the study of how the allocation of resources ...,mankiw_copyright
296,welfare economics,the study of how the allocation of resources ...,mankiw_copyright
297,willingness to pay,the maximum amount that a buyer will pay for ...,mankiw_copyright


In [ ]:
scores = mankiw['definitions'].apply(lambda x: extract_scores(str(x)))
scores_df = pd.DataFrame(list(scores))
scores_df.columns = ['flesch_reading_ease','flesch_kincaid_grade','smog_index','coleman_liau_index','automated_readability_index','dale_chall_readability_score','difficult_words','linsear_write_formula','gunning_fog','text_standard','fernandez_huerta','szigriszt_pazos','gutierrez_polini','crawford','gulpease_index','osman']

mankiw_scores_df = pd.concat([mankiw.reset_index(),scores_df.reset_index()], axis = 1).drop('index', axis = 1)
mankiw_scores_df

,terms,definitions,source,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard,fernandez_huerta,szigriszt_pazos,gutierrez_polini,crawford,gulpease_index,osman
0,absolute advantage,the ability to produce a good using fewer inp...,mankiw_copyright,33.92,11.5,0.0,10.37,7.7,12.13,6,8.0,11.47,11th and 12th grade,80.6,75.43,43.31,4.3,64.833333,63.68
1,accounting profit,total revenue minus total explicit cost,mankiw_copyright,14.63,12.7,0.0,12.05,8.3,14.46,4,3.0,9.07,12th and 13th grade,68.7,65.85,38.13,3.8,82.333333,57.68
2,adverse selection,the tendency for the mix of unobserved attrib...,mankiw_copyright,36.28,12.7,0.0,13.69,13.0,9.79,6,13.0,16.09,12th and 13th grade,80.5,78.08,36.63,4.2,51.777778,45.50
3,agent,a person who is performing an act for another...,mankiw_copyright,49.82,9.5,0.0,9.03,7.5,7.93,2,8.5,11.35,7th and 8th grade,91.6,88.40,45.13,3.3,64.384615,66.69
4,agent,"a person for whom another person, called the ...",mankiw_copyright,66.74,7.2,0.0,8.63,7.5,7.93,1,7.5,8.28,7th and 8th grade,103.6,97.99,45.88,2.6,64.384615,70.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,welfare,government programs that supplement the incom...,mankiw_copyright,45.42,9.2,0.0,14.47,10.3,11.10,3,5.5,8.04,10th and 11th grade,89.7,87.08,36.01,3.0,64.555556,46.57
295,welfare,the study of how the allocation of resources ...,mankiw_copyright,34.93,11.1,0.0,13.73,10.6,9.92,4,7.5,11.67,11th and 12th grade,81.6,76.90,37.56,4.1,59.909091,46.74
296,welfare economics,the study of how the allocation of resources ...,mankiw_copyright,34.93,11.1,0.0,13.73,10.6,9.92,4,7.5,11.67,11th and 12th grade,81.6,76.90,37.56,4.1,59.909091,46.74
297,willingness to pay,the maximum amount that a buyer will pay for ...,mankiw_copyright,77.23,5.2,0.0,3.17,1.6,7.05,2,5.5,8.04,5th and 6th grade,111.6,110.88,55.20,1.4,79.000000,99.50


In [ ]:
mankiw_scores_df.drop_duplicates('terms').to_csv('mankiw.csv', index = False)